In [103]:
import os, sys
import pandas as pd
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
import datetime
from datetime import date
%matplotlib inline

In [104]:
data_dir = '../Data/'

In [120]:
data_train = pd.read_table(data_dir + 'citibike_train.csv', sep='\t', error_bad_lines=False)
data_test = pd.read_csv(data_dir + 'citibike_test.csv', sep='\t', error_bad_lines=False)

/Users/AndreaWenYiWang/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [121]:
data_train.drop(data_test.columns[2:20],axis = 1, inplace= True)
data_test.drop(data_test.columns[2:20],axis = 1, inplace= True)

In [72]:
# def clean_citibike(data):
#     data['datetime'] = pd.to_datetime(data['date'], format='%y-%m-%d')
#     data.drop('date', axis = 1, inplace = True)
#     #binary feature of each day
#     data['date'] = data['datetime'].dt.date
#     data['day_of_week'] = data.datetime.dt.weekday_name
#     data['weekday'] = [0 if x in ['Saturday','Sunday'] else 1 for x in data['day_of_week']]

#     for day in ['Monday', 'Tuesday','Wednesday', 'Thursday','Friday', 'Saturday','Sunday']:
#         data[day] = data.day_of_week == day

#     #holiday    
#     holiday = ['2016-01-01','2016-01-18', '2016-02-15', '2016-05-30',
#                '2016-07-04', '2016-09-05', '2016-10-10', '2016-11-11', '2016-11-24', '2016-12-25']

#     holiday = pd.to_datetime(holiday, format='%Y-%m-%d')
#     data['holiday'] = [1 if x in holiday else 0 for x in data.datetime]

#     #cyclical characteristics of time
#     data['hour_24'] = data.hour + data.pm * 12 
#     data['time_x'] = np.cos((data.hour_24 + data.minute/60) * 2*np.pi /24)
#     data['time_y'] = np.sin((data.hour_24 + data.minute/60) * 2*np.pi /24)
#     data['dayofyear'] = data.datetime.dt.dayofyear
#     data['day_x'] = np.cos((data.dayofyear) * 2*np.pi /365)
#     data['day_y'] = np.sin((data.dayofyear) * 2*np.pi /365)

#     data.drop('datetime', axis = 1, inplace = True)

#     #Target 
#     data['shortage'] = [1 if x<=5 else 0 for x in data['avail_bikes']]
#     return data




In [ ]:
# d_train = clean_citibike(data_train)
# d_test = clean_citibike(data_test)

In [125]:
d_train = data_train.fillna(0)
d_test = data_test.fillna(0)

In [126]:
#X: features
features = ['time_x','time_y','_lat','_long','tot_docks','Monday', 'Tuesday','Wednesday',
 'Thursday','Friday','Saturday','Sunday', 'weekday','day_x','day_y', 'holiday']
X_train = d_train.loc[:,features].as_matrix()
X_test = d_test.loc[:,features].as_matrix()
#Y: target
Y_train = (d_train['shortage']).as_matrix()
Y_test = (d_test['shortage']).as_matrix()


In [127]:
#logistic Regression
from sklearn.linear_model import LogisticRegression as lr
model = lr()
model.fit(X_train,Y_train)


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
((model.predict(X_test) > 0) == Y_test).sum()/X_test.shape[0]

# weather

In [115]:
weather_dir = '../Data/'
weather_filedir = data_dir + 'weather.csv'

weather_data = pd.read_csv(weather_filedir)


weather_features = ['pickup_datetime', 
                    'tempi', #temperature in farenheit
                    'presipi', #presipitation
                    'wspdi', #wind speed in mph
                   'rain', # rain, boolean
                   'snow',# snow, boolean
                   'thunder', # thunder, boolean
                   'visi' ]
weather = weather_data.loc[:,weather_features] #vibilitity in miles



In [116]:
weather.fillna(0, inplace=True)
weather['datetime'] = pd.to_datetime(weather['pickup_datetime'], format='%Y-%m-%d %H:%M::%S', infer_datetime_format=True)
weather['date'] = weather['datetime'].dt.date
weather['hour_24'] = weather['datetime'].dt.hour+1
weather['minute'] = weather['datetime'].dt.minute
weather.drop(['pickup_datetime', 'datetime'],axis = 1, inplace = True)

In [118]:
g = weather[['tempi', 'presipi', 'wspdi', 'rain', 'snow', 'thunder', 'visi']].groupby([weather['date'], weather['hour_24']])
weather_by_hour = g.mean()

for i in ['rain', 'snow', 'thunder']:
    weather_by_hour[i] = [1 if w>0 else 0 for w in weather_by_hour[i]]
    
weather_by_hour.reset_index(inplace = True)

weather_by_hour.head()

,date,hour_24,tempi,presipi,wspdi,rain,snow,thunder,visi
0,2015-12-31,1,46.0,0.0,4.233333,1,0,0,3.833333
1,2015-12-31,2,45.0,0.0,4.600000,0,0,0,8.000000
2,2015-12-31,3,45.0,0.0,0.000000,0,0,0,8.000000
3,2015-12-31,4,44.7,0.0,3.066667,0,0,0,8.333333
4,2015-12-31,5,45.0,0.0,3.500000,0,0,0,8.000000


In [119]:
#joining weather data to original dataset

merged = pd.merge(data,weather_by_hour, on=['date','hour_24'], how = 'left')

In [99]:
merged

,dock_id,dock_name,hour,minute,pm,avail_bikes,avail_docks,tot_docks,_lat,_long,...,day_x,day_y,shortage,tempi,presipi,wspdi,rain,snow,thunder,visi
0,72,W 52 St & 11 Ave,1,12,0,10,27,39,40.767272,-73.993929,...,0.999852,0.017213,0,42.1,0.0,4.6,0.0,0.0,0.0,10.0
1,72,W 52 St & 11 Ave,1,24,0,10,27,39,40.767272,-73.993929,...,0.999852,0.017213,0,42.1,0.0,4.6,0.0,0.0,0.0,10.0
2,72,W 52 St & 11 Ave,1,35,0,10,27,39,40.767272,-73.993929,...,0.999852,0.017213,0,42.1,0.0,4.6,0.0,0.0,0.0,10.0
3,72,W 52 St & 11 Ave,1,46,0,10,27,39,40.767272,-73.993929,...,0.999852,0.017213,0,42.1,0.0,4.6,0.0,0.0,0.0,10.0
4,72,W 52 St & 11 Ave,1,57,0,9,28,39,40.767272,-73.993929,...,0.999852,0.017213,0,42.1,0.0,4.6,0.0,0.0,0.0,10.0
5,72,W 52 St & 11 Ave,2,9,0,9,28,39,40.767272,-73.993929,...,0.999852,0.017213,0,41.0,0.0,3.5,0.0,0.0,0.0,10.0
6,72,W 52 St & 11 Ave,2,20,0,9,28,39,40.767272,-73.993929,...,0.999852,0.017213,0,41.0,0.0,3.5,0.0,0.0,0.0,10.0
7,72,W 52 St & 11 Ave,2,31,0,9,28,39,40.767272,-73.993929,...,0.999852,0.017213,0,41.0,0.0,3.5,0.0,0.0,0.0,10.0
8,72,W 52 St & 11 Ave,2,42,0,9,28,39,40.767272,-73.993929,...,0.999852,0.017213,0,41.0,0.0,3.5,0.0,0.0,0.0,10.0
9,72,W 52 St & 11 Ave,2,53,0,9,28,39,40.767272,-73.993929,...,0.999852,0.017213,0,41.0,0.0,3.5,0.0,0.0,0.0,10.0


In [ ]:
#X: features
feature = features + weather_features
X_train = d_train.loc[:,features].as_matrix()
X_test = d_test.loc[:,features].as_matrix()
#Y: target
Y_train = (d_train['shortage']).as_matrix()
Y_test = (d_test['shortage']).as_matrix()


In [ ]:
#logistic Regression
from sklearn.linear_model import LogisticRegression as lr
model = lr()
model.fit(X_train,Y_train)


In [ ]:
((model.predict(X_test) > 0) == Y_test).sum()/X_test.shape[0]